In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier


#from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.utils import class_weight

import numpy as np
import pandas as pd



data = pd.read_csv('Train_clean.csv')

In [ ]:
class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [ ]:
models1 = {
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
}

params1 = {
    'ExtraTreesClassifier': {'n_estimators': [16, 32] },
    'RandomForestClassifier': {'n_estimators': [16, 32] },
    'AdaBoostClassifier':  {'n_estimators': [16, 32] },
    'GradientBoostingClassifier': {'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] }
}

In [ ]:
models2 = {
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'SVC': SVC(),
}

params2 = {
    'ExtraTreesClassifier': {'n_estimators': [16, 32] },
    'RandomForestClassifier': {'n_estimators': [16, 32] },
    'AdaBoostClassifier':  {'n_estimators': [16, 32] },
    'GradientBoostingClassifier': {'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] },
    'SVC': [
        {'kernel': ['linear'], 'C': [1, 10, 100]},
        {'kernel': ['rbf', 'poly', 'sigmoid', 'linear'], 'C':[0.1, 1, 10, 100], 'gamma': [1,0.1,0.01,0.001]},
    ]
}

In [2]:
X = data.drop(columns=['Survived', "Unnamed: 0"], axis=1)
y = data.Survived
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=0)


In [5]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [4]:
pred = clf.predict(X_test_norm)

clf.score(X_test_norm, y_test)

NameError: name 'clf' is not defined

In [ ]:
10.0 ** -np.arange(1, 7)

In [ ]:
helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(X_train_norm, y_train, scoring='f1', n_jobs=2)

Running GridSearchCV for ExtraTreesClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits
/shared-libs/python3.7/py/lib/python3.7/site-packages/joblib/externals/loky/backend/resource_tracker.py:120: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some folders/sempahores might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '
exception calling callback for <Future at 0x7fc790ab3590 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/joblib/parallel.py", line 792, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/shared-libs/

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {EXIT(1)}

In [ ]:
display(helper1.score_summary(sort_by='max_score').head(20))

In [ ]:
'LogReg': LogisticRegression()
'LinReg': LinearRegression()
'KNeighborsClassifier': KNeighborsClassifier()

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train_norm, y_train)

In [8]:
tuned_parameters = [{'alpha': list(10.0 ** -np.arange(1, 7)), 'activation': ('identity', 'logistic', 'tanh', 'relu'),
                     'random_state': [0], 'verbose' : [3]}]

scores = ['precision', 'recall']


for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        MLPClassifier(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train_norm, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test_norm)
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for precision

Iteration 1, loss = 0.77481206
Iteration 2, loss = 0.71851731
Iteration 3, loss = 0.66705555
Iteration 4, loss = 0.62726439
Iteration 5, loss = 0.59526732
Iteration 6, loss = 0.56771339
Iteration 7, loss = 0.54434934
Iteration 8, loss = 0.52595798
Iteration 9, loss = 0.50867696
Iteration 10, loss = 0.49471355
Iteration 11, loss = 0.48181174
Iteration 12, loss = 0.47126835
Iteration 13, loss = 0.46111227
Iteration 14, loss = 0.45320070
Iteration 15, loss = 0.44627017
Iteration 16, loss = 0.44053605
Iteration 17, loss = 0.43560650
Iteration 18, loss = 0.43229979
Iteration 19, loss = 0.42867448
Iteration 20, loss = 0.42607099
Iteration 21, loss = 0.42403569
Iteration 22, loss = 0.42198758
Iteration 23, loss = 0.42090920
Iteration 24, loss = 0.41969361
Iteration 25, loss = 0.41905116
Iteration 26, loss = 0.41846761
Iteration 27, loss = 0.41787796
Iteration 28, loss = 0.41749237
Iteration 29, loss = 0.41720145
Iteration 30, loss = 0.41696517
Iteratio

Iteration 10, loss = 0.51024638
Iteration 11, loss = 0.49773632
Iteration 12, loss = 0.48748271
Iteration 13, loss = 0.47805327
Iteration 14, loss = 0.47061706
Iteration 15, loss = 0.46390349
Iteration 16, loss = 0.45862325
Iteration 17, loss = 0.45407358
Iteration 18, loss = 0.45077836
Iteration 19, loss = 0.44740142
Iteration 20, loss = 0.44487804
Iteration 21, loss = 0.44281006
Iteration 22, loss = 0.44103996
Iteration 23, loss = 0.44010134
Iteration 24, loss = 0.43893616
Iteration 25, loss = 0.43832996
Iteration 26, loss = 0.43758014
Iteration 27, loss = 0.43724675
Iteration 28, loss = 0.43685227
Iteration 29, loss = 0.43639130
Iteration 30, loss = 0.43634936
Iteration 31, loss = 0.43596923
Iteration 32, loss = 0.43602614
Iteration 33, loss = 0.43582167
Iteration 34, loss = 0.43560795
Iteration 35, loss = 0.43552155
Iteration 36, loss = 0.43549766
Iteration 37, loss = 0.43539098
Iteration 38, loss = 0.43536448
Iteration 39, loss = 0.43528596
Iteration 40, loss = 0.43538001
Iteratio

Iteration 31, loss = 0.42271373
Iteration 32, loss = 0.42236700
Iteration 33, loss = 0.42249805
Iteration 34, loss = 0.42198032
Iteration 35, loss = 0.42202106
Iteration 36, loss = 0.42171855
Iteration 37, loss = 0.42175027
Iteration 38, loss = 0.42162168
Iteration 39, loss = 0.42140576
Iteration 40, loss = 0.42149729
Iteration 41, loss = 0.42133050
Iteration 42, loss = 0.42139696
Iteration 43, loss = 0.42114638
Iteration 44, loss = 0.42137917
Iteration 45, loss = 0.42115632
Iteration 46, loss = 0.42105628
Iteration 47, loss = 0.42098879
Iteration 48, loss = 0.42112466
Iteration 49, loss = 0.42098999
Iteration 50, loss = 0.42099944
Iteration 51, loss = 0.42101994
Iteration 52, loss = 0.42108647
Iteration 53, loss = 0.42093959
Iteration 54, loss = 0.42086487
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.78124098
Iteration 2, loss = 0.72670591
Iteration 3, loss = 0.68193662
Iteration 4, loss = 0.64293232
Iteration 5, loss 

Iteration 1, loss = 0.77257477
Iteration 2, loss = 0.71949083
Iteration 3, loss = 0.67434640
Iteration 4, loss = 0.63761353
Iteration 5, loss = 0.60693579
Iteration 6, loss = 0.58139214
Iteration 7, loss = 0.56085485
Iteration 8, loss = 0.54197902
Iteration 9, loss = 0.52610355
Iteration 10, loss = 0.51163843
Iteration 11, loss = 0.49973355
Iteration 12, loss = 0.48972548
Iteration 13, loss = 0.48097510
Iteration 14, loss = 0.47314422
Iteration 15, loss = 0.46760778
Iteration 16, loss = 0.46243478
Iteration 17, loss = 0.45903819
Iteration 18, loss = 0.45549335
Iteration 19, loss = 0.45253967
Iteration 20, loss = 0.45071753
Iteration 21, loss = 0.44920328
Iteration 22, loss = 0.44752732
Iteration 23, loss = 0.44654633
Iteration 24, loss = 0.44573689
Iteration 25, loss = 0.44512914
Iteration 26, loss = 0.44454590
Iteration 27, loss = 0.44419305
Iteration 28, loss = 0.44411366
Iteration 29, loss = 0.44374520
Iteration 30, loss = 0.44349943
Iteration 31, loss = 0.44343395
Iteration 32, los

Iteration 11, loss = 0.47576904
Iteration 12, loss = 0.46520940
Iteration 13, loss = 0.45503692
Iteration 14, loss = 0.44711026
Iteration 15, loss = 0.44016694
Iteration 16, loss = 0.43442301
Iteration 17, loss = 0.42948361
Iteration 18, loss = 0.42616971
Iteration 19, loss = 0.42253639
Iteration 20, loss = 0.41992503
Iteration 21, loss = 0.41788358
Iteration 22, loss = 0.41583077
Iteration 23, loss = 0.41474773
Iteration 24, loss = 0.41352945
Iteration 25, loss = 0.41288496
Iteration 26, loss = 0.41230260
Iteration 27, loss = 0.41171190
Iteration 28, loss = 0.41132824
Iteration 29, loss = 0.41104021
Iteration 30, loss = 0.41080683
Iteration 31, loss = 0.41055664
Iteration 32, loss = 0.41061577
Iteration 33, loss = 0.41042792
Iteration 34, loss = 0.41025235
Iteration 35, loss = 0.41023123
Iteration 36, loss = 0.41019344
Iteration 37, loss = 0.41011139
Iteration 38, loss = 0.41006087
Iteration 39, loss = 0.40999552
Iteration 40, loss = 0.41004203
Iteration 41, loss = 0.41003849
Iteratio

...

/Users/tobiasschulz/opt/anaconda3/envs/ml_3_7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 12, loss = 0.52466568
Iteration 13, loss = 0.51516566
Iteration 14, loss = 0.50581005
Iteration 15, loss = 0.49838164
Iteration 16, loss = 0.49147935
Iteration 17, loss = 0.48561914
Iteration 18, loss = 0.47980215
Iteration 19, loss = 0.47442556
Iteration 20, loss = 0.46994640
Iteration 21, loss = 0.46566891
Iteration 22, loss = 0.46168421
Iteration 23, loss = 0.45792368
Iteration 24, loss = 0.45467627
Iteration 25, loss = 0.45156657
Iteration 26, loss = 0.44883647
Iteration 27, loss = 0.44628536
Iteration 28, loss = 0.44394534
Iteration 29, loss = 0.44178042
Iteration 30, loss = 0.43974813
Iteration 31, loss = 0.43797908
Iteration 32, loss = 0.43630070
Iteration 33, loss = 0.43476119
Iteration 34, loss = 0.43318100
Iteration 35, loss = 0.43178625
Iteration 36, loss = 0.43064800
Iteration 37, loss = 0.42935131
Iteration 38, loss = 0.42821865
Iteration 39, loss = 0.42713679
Iteration 40, loss = 0.42619639
Iteration 41, loss = 0.42518010
Iteration 42, loss = 0.42425635
Iteratio

/Users/tobiasschulz/opt/anaconda3/envs/ml_3_7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 17, loss = 0.43690787
Iteration 18, loss = 0.43512589
Iteration 19, loss = 0.43330641
Iteration 20, loss = 0.43226147
Iteration 21, loss = 0.43090152
Iteration 22, loss = 0.43037120
Iteration 23, loss = 0.42946553
Iteration 24, loss = 0.42914785
Iteration 25, loss = 0.42851544
Iteration 26, loss = 0.42838009
Iteration 27, loss = 0.42773453
Iteration 28, loss = 0.42735891
Iteration 29, loss = 0.42741573
Iteration 30, loss = 0.42718325
Iteration 31, loss = 0.42692798
Iteration 32, loss = 0.42673820
Iteration 33, loss = 0.42662256
Iteration 34, loss = 0.42643980
Iteration 35, loss = 0.42622803
Iteration 36, loss = 0.42608570
Iteration 37, loss = 0.42578586
Iteration 38, loss = 0.42586007
Iteration 39, loss = 0.42556464
Iteration 40, loss = 0.42535913
Iteration 41, loss = 0.42547327
Iteration 42, loss = 0.42549353
Iteration 43, loss = 0.42515784
Iteration 44, loss = 0.42490734
Iteration 45, loss = 0.42470952
Iteration 46, loss = 0.42460997
Iteration 47, loss = 0.42451747
Iteratio

/Users/tobiasschulz/opt/anaconda3/envs/ml_3_7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
tuned_parameters = [{'solver': ['lbfgs'], 'alpha': list(10.0 ** -np.arange(1, 7)), 'activation': ('identity', 'logistic', 'tanh', 'relu'),
                     'random_state': [0]}]

scores = ['precision', 'recall']


for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        MLPClassifier(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train_norm, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test_norm)
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for precision



/Users/tobiasschulz/opt/anaconda3/envs/ml_3_7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/tobiasschulz/opt/anaconda3/envs/ml_3_7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/tobiasschulz/opt/anaconda3/envs/ml_3_7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perce

/Users/tobiasschulz/opt/anaconda3/envs/ml_3_7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/tobiasschulz/opt/anaconda3/envs/ml_3_7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/tobiasschulz/opt/anaconda3/envs/ml_3_7/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perce